In [1]:
import time
import pandas as pd
from wikiapi import WikiApi
wiki = WikiApi()
import re
from gensim.models import Word2Vec
#import word2vec
from scipy.spatial.distance import cosine
import numpy as np
import heapq
import string


In [9]:


def remove_punctuation(s):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    return s
#TODO: fix this or dont use
def get_longword(s, n):
    s = re.split(' ', s)
    return heapq.nlargest(n, s, key=len)

#TODO: make this work with one word/period
def get_avg_vec(words):
	try:
		words = re.split(' ', words)
	except:
		pass
	nwords = 0
	featureVec = np.zeros((num_features,))#,dtype="float32")
	for word in words:
		if word in index2word_set:
			nwords = nwords + 1.
			featureVec = np.add(featureVec,model[word])
	if nwords > 0:
		featureVec = np.divide(featureVec,nwords)
	return featureVec

def get_tvec(row):
    if row['correctAnswer'] == 'A':
        a = row['aavec']
        return a
    elif row['correctAnswer'] == 'B':
        a = row['bavec']
        return a
    elif row['correctAnswer'] == 'C':
        a = row['cavec']
        return a
    elif row['correctAnswer'] == 'D':
        a = row['davec']
        return a
    
def run_through_theta(v):
    return v.dot(theta.as_matrix())

def get_winner_from_avecs(row):
	try:	
		dists = []
		for col in row['answerA':'answerD']:
		#TODO: if 'not' in words: 1-avec
			avec = get_avg_vec(col)
			#if 'not' in re.split(' ',col):
			#	avec = 1-avec
			#	print('NOT')
			#why nan errors?
			if sum(np.isfinite(avec)) < 300:
				print('avec')
				print(avec)
			if  sum(np.isfinite(row['pvec'])) < 300:
				print('pvec')
				print(row['pvec'])
			dist = cosine(avec, row['pvec'])
			dists.append(dist)
		m = np.nanmin(dists)
		best = [i for i, j in enumerate(dists) if j == m]
		if best == [0]:
			return 'A'
		if best == [1]:

			return 'B'
		if best == [2]:
			return 'C'
		if best == [3]:
			return 'D'
		else:
			print('returning C becuase no best cosine')
			print(len(best))
			print(dists)
			print(m)
			return 'C'
	except:
		print('some error, returning C')
		return 'C'
    
    
    
    
def word_clusters(row):
    try:
        keywds = row.keyword

        word_pos = {}
        for keyw in keywds:
            ls = [i for i,x in enumerate(words) if x == keyw]
            #print(ls)
            word_pos[keyw] = ls

        spots = []
        for key1 in keywds:
            matches = {}
            for x in word_pos[key1]:
                dists=[]
                for key in keywds:
                    temp_pos = 0
                    temp_dist = 10000000000
                    for i in word_pos[key]:
                    #print(i)
                        if abs(x-i)<temp_dist:
                            temp_dist = abs(x-i)
                            temp_pos = i
                    dists.append(temp_dist)
                dists.remove(max(dists))
                matches[x] = sum(dists)
            #print(matches)
            #print(matches)
            zzz= [k for k, v in matches.items() if v ==min(matches.values())]
            spots.append(zzz[0])

        answers = row['akeyword':'dkeyword']
        kkk = []
        for answer in answers:
            waka = 0
            for x in spots:
                wd = words[x-1000:x+1000]
                ct = wd.count(answer[0])
                waka = waka + ct
            kkk.append(waka)
        print(kkk)
        mx = max(kkk)
        best = [i for i, j in enumerate(kkk) if j == mx]
        if best == [0]:
            return 'A'
        if best == [1]:
            return 'B'
        if best == [2]:
            return 'C'
        if best == [3]:
            return 'D'
        else:
            print('returning C becuase no max')
            return 'C'
    except:
        print('returning C becuase of BIG PROBLEM')
        return 'C'

#DOnt use
start = time.time()
model = Word2Vec.load_word2vec_format('/Users/liamconnell/Downloads/GoogleNews-vectors-negative300.bin', binary = True)
lap1 = time.time()
print('data gathered: %s' % (lap1 - start))

index2word_set = set(model.index2word)
num_features = 300

In [10]:
data_all  = pd.read_csv('../input/training_set.tsv', '\t')
train = data_all.sample(frac=0.9, random_state=1)
data = data_all.loc[~data_all.index.isin(train.index)]

In [11]:
file = open('../input/Concepts.txt', 'r')
words = list(file.read().split())
words = [word.strip(string.punctuation) for word in words]

In [12]:
data.question = data.question.apply(remove_punctuation)
data.answerA = data.answerA.apply(remove_punctuation)
data.answerB = data.answerB.apply(remove_punctuation)
data.answerC = data.answerC.apply(remove_punctuation)
data.answerD = data.answerD.apply(remove_punctuation)

/Users/liamconnell/anaconda/lib/python3.4/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [13]:
data['keyword'] =data['question'].apply(get_longword, args = (5,))



/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
#data['qvec'] = data.question.apply(get_avg_vec) 
data['akeyword'] =data['answerA'].apply(get_longword, args = (1,))
data['bkeyword'] =data['answerB'].apply(get_longword, args = (1,))
data['ckeyword'] =data['answerC'].apply(get_longword, args = (1,))
data['dkeyword'] =data['answerD'].apply(get_longword, args = (1,))



/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [15]:
wd = words[233322-1000:233322+1000]
wd.count('meters')

2

In [16]:
data['guess'] = data.apply(word_clusters, axis = 1)

[0, 27, 0, 1]
[9, 5, 0, 0]
[0, 0, 2, 14]
[0, 0, 0, 0]
returning C becuase no max
[0, 0, 23, 23]
returning C becuase no max
[0, 0, 0, 0]
returning C becuase no max
[0, 0, 0, 0]
returning C becuase no max
[0, 3, 0, 0]
[3, 0, 3, 2]
returning C becuase no max
[0, 9, 0, 0]
[0, 0, 0, 0]
returning C becuase no max
[5, 0, 0, 0]
returning C becuase of BIG PROBLEM
[4, 4, 0, 3]
returning C becuase no max
[0, 0, 9, 9]
returning C becuase no max
[12, 70, 28, 23]
returning C becuase of BIG PROBLEM
[0, 4, 0, 3]
[0, 55, 55, 55]
returning C becuase no max
[0, 0, 0, 0]
returning C becuase no max
[5, 124, 0, 2]
[0, 0, 0, 0]
returning C becuase no max
[0, 1, 3, 0]
[0, 28, 0, 12]
[0, 1, 1, 2]
[0, 0, 0, 0]
returning C becuase no max
[0, 0, 0, 0]
returning C becuase no max
[4, 0, 2, 2]
[13, 2, 64, 14]
[0, 0, 12, 0]
[88, 88, 88, 88]
returning C becuase no max
[2, 0, 0, 0]
[0, 0, 212, 0]
[16, 4, 0, 4]
[7, 7, 0, 0]
returning C becuase no max
[0, 1, 0, 16]
[3, 4, 2, 0]
[0, 0, 0, 0]
returning C becuase no max
[0,

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
data['guess'] == data['correctAnswer']

15       True
20      False
25      False
63      False
77      False
86       True
96      False
100      True
109      True
129     False
143      True
144     False
145     False
150     False
152     False
155     False
183      True
222     False
235     False
243     False
252     False
253      True
266     False
269     False
290     False
313     False
316      True
319     False
324     False
327      True
        ...  
2230    False
2244    False
2247    False
2254    False
2257    False
2258    False
2267     True
2277    False
2283     True
2299    False
2311    False
2326    False
2327    False
2342    False
2365     True
2370    False
2376    False
2404    False
2446    False
2448    False
2452    False
2458    False
2461     True
2465    False
2476    False
2480    False
2482    False
2483     True
2485    False
2487     True
dtype: bool

In [ ]:
sample = pd.read_csv('../input/sample_submission.csv')
sample['correctAnswer'] = data['guess']
sample.to_csv('../output/word_cluster_00.csv', index=False)

In [9]:
data.head()

,id,question,answerA,answerB,answerC,answerD,keyword,qvec,akeyword,bkeyword,ckeyword,dkeyword,aavec,bavec,cavec,davec
0,102501,A meter is industry by developing a process of...,cool foods during shipping,yard,nervous system,birds of prey,"[developing, industry, examples, process, heat...","[-0.0076541421935, 0.0428126163781, -0.0093698...",[shipping],[yard],[nervous],[birds],"[0.0599160343409, 0.123002231121, -0.045650314...","[0.00865275319666, 0.0433680191636, 0.04642602...","[0.0429564490914, 0.00291655748151, -0.0250321...","[0.0218830313534, 0.0545616932213, -0.10153727..."
1,102502,Which part of directly involved in recycling t...,panting,urethra,predators,water,"[reproduction, temperature, recycling, directl...","[-0.00762585951015, 0.00861827973276, -0.01647...",[panting],[urethra],[predators],[water],"[0.0639518499374, 0.0659503489733, -0.02526669...","[-0.0568876340985, 0.0671530738473, -0.1180525...","[-0.0123091973364, 0.0607602931559, -0.1267585...","[-0.0583994314075, 0.0527478717268, 0.04502407..."
2,102503,Which of the seeds grow better in the light or...,muscular system,keep one group in a light place and another in...,skates and rays,nervous tissue,"[framework, provides, better, organs, Which]","[-0.0406858625356, 0.000434694252908, 0.027284...",[muscular],[another],[skates],[nervous],"[0.0851534381509, 0.12938041985, 0.03531557321...","[0.119020909071, -0.0120366625488, 0.056220933...","[-0.0518872700632, -0.0753730908036, 0.0167950...","[0.0429564490914, 0.00291655748151, -0.0250321..."
3,102504,Which of the hermaphroditic This growing seaso...,coniferous forest,no reproductive organs,there is less water loss through small leaves,perennial,"[hermaphroditic, growing, season, Which, think]","[-0.0035101480782, 0.0597439706326, -0.0213354...",[coniferous],[reproductive],[through],[perennial],"[0.0473578386009, 0.0444385223091, 0.017191544...","[0.00988108664751, 0.0588776506484, -0.0523356...","[0.00783920008689, 0.0335110835731, -0.0832989...","[0.0817415639758, 0.203823119402, 0.0391014851..."
4,102505,When marsupials following is not a transport o...,heart,glucagon,hatched,to protect vital organs,"[marsupials, following, transport, response, f...","[0.00498434053734, 0.000841771054547, -0.01591...",[heart],[glucagon],[hatched],[protect],"[-0.0549635440111, 0.0888696238399, 0.02337735...","[-0.0631601214409, -0.0484089106321, -0.010907...","[-0.0236770678312, 0.0858293697238, -0.0198295...","[-0.0634209290147, 0.0630781129003, -0.0521080..."
